In [1]:
import lsst.meas.astrom as measAstrom
import lsst.daf.base as dafBase
import lsst.pipe.base as pipeBase
import lsst.daf.persistence as dafPersist

import lsst.afw.display.ds9 as ds9
import DiaSourceTools as DSTools

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom

import numpy as np

/renoir_data_00/fouchez/lsst/DM/stacks/lsstsw/2016-04-04/lsstsw/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output")

In [54]:
diffExp1 = butler.get("deepDiff_differenceExp", {'visit': 849687 , 'filter':'r' , 'ccd':14})
diaSrcs1 = butler.get("deepDiff_diaSrc", {'visit': 849687 , 'filter':'r' , 'ccd':14})

In [55]:
diffExp2 = butler.get("deepDiff_differenceExp", {'visit': 850177 , 'filter':'r' , 'ccd':14})
diaSrcs2 = butler.get("deepDiff_diaSrc", {'visit': 850177 , 'filter':'r' , 'ccd':14})

In [56]:

def get_good_diasrcs(diaSrcs, diffExp):
    class_diaSrcs = diaSrcs.copy()
    class_diaSrcs.clear()
    
    fluxes, sigmas = DSTools.get_fluxes_and_sigmas(diaSrcs, "base_PeakLikelihoodFlux_flux")

    m_sigma = np.mean(sigmas)
    t_sigma = 5*m_sigma

    sources = DSTools.get_sources_over_sigma(diaSrcs, t_sigma, "base_PeakLikelihoodFlux_flux")

    for source in sources:
                try:
                    stamp = DSTools.get_stamp(source, diffExp)
                    stamp_clone = stamp.clone()

                    results = DSTools.detect_diasources(stamp_clone, doSmooth=True)
                    negFps = len(results.fpSets.negative.getFootprints())
                    posFps = len(results.fpSets.positive.getFootprints())

                    #if negFps == 0:
                    if True:
                        class_diaSrcs.addNew()
                        class_diaSrcs.append(source)


                except Exception, e:
                    pass
    return class_diaSrcs

In [57]:
c_ds1= get_good_diasrcs(diaSrcs1, diffExp1)
c_ds2= get_good_diasrcs(diaSrcs2, diffExp2)

In [60]:
matches = afwTable.matchRaDec(c_ds1, c_ds2, afwGeom.Angle(1./3600., afwGeom.degrees)) 

In [58]:
ds9.mtv(diffExp1, frame=1)
ds9.setMaskTransparency(100, frame=1)
ds9.mtv(diffExp2, frame=2)
ds9.setMaskTransparency(100, frame=2)

ds9 doesn't appear to be running (XPA: XPA returned NULL, (;iconify no; raise)), I'll try to exec it for you


In [31]:
print len(matches)

198


In [59]:
counter = 0
for src in c_ds1:
    ds9.dot('o', src.getX(), src.getY(), frame=1,size=10, ctype=ds9.RED)
for src in c_ds2:
    ds9.dot('o', src.getX(), src.getY(), frame=2,size=10, ctype=ds9.RED)

In [62]:
for match in matches:
    ds9.dot('o', match.first.getX(), match.first.getY(), frame=1,size=12)
    ds9.dot('o', match.second.getX(), match.second.getY(), frame=2, size=12)

In [65]:
a = [1, 2, 3]

for elem in a:
    print [a==elem]

[False]
[False]
[False]


In [67]:
a == 2

False